In [4]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [129]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import sqlite3 as db
import sys
import json

conn = db.connect("pgatour_stats.db")
c = conn.cursor()

<IPython.core.display.Javascript object>

##### Get stats for each player from 2014-2019

In [123]:
year_list = ["2014", "2015", "2016", "2017", "2018", "2019"]

#  Dictionary of stats and their URLs
url_dict = {
    "scoring": "https://www.pgatour.com/stats/stat.120.y",
    "driving_distance": "https://www.pgatour.com/stats/stat.101.y",
    "good_drives": "https://www.pgatour.com/stats/stat.02438.y",
    "right_tendency": "https://www.pgatour.com/content/pgatour/stats/stat.02423.y",
    "left_tendency": "https://www.pgatour.com/content/pgatour/stats/stat.02422.y",
    "gir": "https://www.pgatour.com/content/pgatour/stats/stat.103.y",
    "gir_<100": "https://www.pgatour.com/stats/stat.02330.y",
    "gir_100-125": "https://www.pgatour.com/content/pgatour/stats/stat.077.y",
    "gir_125-150": "https://www.pgatour.com/stats/stat.329.y",
    "gir_150-175": "https://www.pgatour.com/stats/stat.328.y",
    "gir_175-200": "https://www.pgatour.com/stats/stat.327.y",
    "gir_<200": "https://www.pgatour.com/stats/stat.326.y",
    "going_for_the_green": "https://www.pgatour.com/content/pgatour/stats/stat.419.y",
    "sand_saves": "https://www.pgatour.com/stats/stat.111.y",
    "scrambling": "https://www.pgatour.com/stats/stat.130.y",
    "one_putts": "https://www.pgatour.com/content/pgatour/stats/stat.413.y",
    "three_putts": "https://www.pgatour.com/content/pgatour/stats/stat.426.y",
}

for year in year_list:
    final_df = pd.DataFrame()
    for category, url in url_dict.items():
        print(category)
        stat_url = url + year + ".html"
        
        # Create soup
        response = requests.get(stat_url)
        text = response.text
        soup = BeautifulSoup(text, "lxml")

        # Create players list
        player_list = []
        players = soup.select("td a")[1:]
        for player in players:
            player_list.append(player.get_text())

        # Create list of stat headers
        headers = []
        rounds = soup.find_all(class_="rounds hidden-small hidden-medium")[0].get_text()
        headers.append(rounds)
        stat_headers = soup.find_all(class_="col-stat")
        for header in stat_headers:
            headers.append(header.get_text())

        # Create list of stats
        stats = soup.find_all("tr")[2:]
        stats_list = []
        for elem in stats:
            small_list = []
            for tag in elem.find_all("td")[3:]:
                stat = tag.get_text()
                small_list.append(stat)
            stats_list.append(small_list)

        # Create dictionaries of player and stat dictionary
        count = 0
        player_dict = dict()
        for player in player_list:
            stats_dict = dict()
            for stat in stats_list[count]:
                stats_dict[
                    category + "_" + headers[stats_list[count].index(stat)]
                ] = stat
            player_dict[player] = stats_dict
            count += 1
        
        # Create dataframe
        df = pd.DataFrame.from_dict(player_dict, orient="index")

        if len(final_df) == 0:
            final_df = df
        else:
            final_df = final_df.join(df, how="outer")

    # Write to database
    final_df.to_sql(year, conn, if_exists="replace")
    print("~" * 64, "\n")

scoring
driving_distance
good_drives
right_tendency
left_tendency
gir
gir_<100
gir_100-125
gir_125-150
gir_150-175
gir_175-200
gir_<200
going_for_the_green
sand_saves
scrambling
one_putts
three_putts
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 

scoring
driving_distance
good_drives
right_tendency
left_tendency
gir
gir_<100
gir_100-125
gir_125-150
gir_150-175
gir_175-200
gir_<200
going_for_the_green
sand_saves
scrambling
one_putts
three_putts
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 

scoring
driving_distance
good_drives
right_tendency
left_tendency
gir
gir_<100
gir_100-125
gir_125-150
gir_150-175
gir_175-200
gir_<200
going_for_the_green
sand_saves
scrambling
one_putts
three_putts
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 

scoring
driving_distance
good_drives
right_tendency
left_tendency
gir
gir_<100
gir_100-125
gir_125-150
gir_150-175
gir_175-200
gir_<200
going_for_the_green
sand_saves
scrambling
one_putts
three_putts

<IPython.core.display.Javascript object>

In [86]:
# Dictionary of stat IDs and names
stat_id_dict = dict()
stat_list = tournament_data["statList"]

for elem in stat_list:
    stat_id_dict[elem["stid"]] = elem["stName"]

stat_id_dict

{'103': 'Greens in Regulation Percentage',
 '102': 'Driving Accuracy Percentage',
 '159': 'Longest Drive',
 '101': 'Driving Distance',
 '129': 'Total Driving',
 '111': 'Sand Save Percentage',
 '127': 'All-Around Ranking',
 '158': 'Ball Striking',
 '02564': 'SG: Putting',
 '171': 'Par 3 Performance',
 '172': 'Par 4 Performance',
 '173': 'Par 5 Performance',
 '207': 'Front 9 Scoring Average',
 '208': 'Back 9 Scoring Average'}

<IPython.core.display.Javascript object>

##### Create list of tournaments played from 2014-2019

In [90]:
year_list = ["2014", "2015", "2016", "2017", "2018", "2019"]
tournament_list = []
for year in year_list:
    url = "https://www.pgatour.com/stats/stat.02671.y" + year + ".eoff.t060.html"

    response = requests.get(url)
    text = response.text
    soup = BeautifulSoup(text, "lxml")

    players = soup.find_all(class_="player-name")[1:]

    player_id_dict = dict()

    for player in players:
        player_href = player.find("a")["href"]
        player_name = player.find("a").get_text()

        player_id = ""

        for ch in player_href:
            if ch.isdigit():
                player_id += ch

        player_id_dict[player_name] = player_id

    for k, v in player_id_dict.items():
        tournament_data = json.loads(
            requests.get(
                "https://statdata.pgatour.com/players/" + v + "/r_recap.json"
            ).text
        )

        for i in tournament_data["plr"]["tours"][0]["years"]:
            if i["year"] == year:
                for tournament in i["trnDetails"]:
                    if tournament["trn"]["trnName"] not in tournament_list:
                        tournament_list.append(tournament["trn"]["trnName"])

tournament_list

['Frys.com Open',
 'CIMB Classic',
 'WGC-HSBC Champions',
 "(*) Wendy's 3-Tour Challenge",
 '(*) Franklin Templeton',
 'Hyundai T of C',
 'Humana Challenge',
 'Farmers Insurance Open',
 'Waste Management Phoenix',
 'WGC-Accenture Match Play',
 'The Honda Classic',
 'WGC-Cadillac Championship',
 "Arnold Palmer Invt'l",
 'Valero Texas Open',
 'Masters Tournament',
 'RBC Heritage',
 'Zurich Classic of New Orl',
 'THE PLAYERS Championship',
 'the Memorial Tournament',
 'FedEx St. Jude Classic',
 'U.S. Open',
 '(*) CVS Caremark Charity',
 'Quicken Loans National',
 'The Open Championship',
 'PGA Championship',
 'Wyndham Championship',
 'The Barclays',
 'Deutsche Bank Champ',
 'BMW Championship',
 'TOUR Championship',
 'Shriners Hospitals',
 'The McGladrey Classic',
 'Sony Open in Hawaii',
 'AT&T Pebble Beach',
 'Shell Houston Open',
 'Wells Fargo Championship',
 'Crowne Plaza Invitational',
 'The Greenbrier Classic',
 'John Deere Classic',
 "WGC-Bridgestone Invit'l",
 '(*) World Challenge',

<IPython.core.display.Javascript object>

##### Get player stats for each tournament

In [119]:
year_list = ["2014", "2015", "2016", "2017", "2018", "2019"]

tournament_dict_list = list()
for tournament in tournament_list:
    tournament_dict = dict()
    player_dict_list = list()
    for year in year_list:
        url = "https://www.pgatour.com/stats/stat.02671.y" + year + ".eoff.t060.html"

        response = requests.get(url)
        text = response.text
        soup = BeautifulSoup(text, "lxml")

        players = soup.find_all(class_="player-name")[1:]

        player_id_dict = dict()

        for player in players:
            player_href = player.find("a")["href"]
            player_name = player.find("a").get_text()

            player_id = ""

            for ch in player_href:
                if ch.isdigit():
                    player_id += ch

            player_id_dict[player_name] = player_id

        for k, v in player_id_dict.items():
            tournament_data = json.loads(
                requests.get(
                    "https://statdata.pgatour.com/players/" + v + "/r_recap.json"
                ).text
            )

            player_dict = dict()
            for i in tournament_data["plr"]["tours"][0]["years"]:
                if i["year"] == year:
                    for elem in i["trnDetails"]:
                        if elem["trn"]["trnName"] == tournament:
                            player_dict["Name"] = k
                            player_dict["Finish"] = elem["finPos"]["fmt"]
                            player_dict["Score"] = elem["scr"]["relToPar"]
                            player_dict["Average Score"] = elem["scrAvg"]

                            for d in elem["profiles"][0]["stats"]:
                                player_dict[stat_id_dict[d["stid"]]] = d["stValue"]

                            player_dict["Year"] = year
                            player_dict["Date"] = elem["endDate"]

            if len(player_dict) > 0:
                player_dict_list.append(player_dict)
                print("...")

    tournament_dict[tournament] = player_dict_list
    tournament_dict_list.append(tournament_dict)

...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...


...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...


...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...


...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...


...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...


...

...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...


...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...


...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...


...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...


<IPython.core.display.Javascript object>

##### Write tournament tables to database

In [130]:
for elem in tournament_dict_list:
    for k, v in elem.items():
        df = pd.DataFrame(v)
        df.to_sql(k, conn, if_exists="replace")
        print(k)

/opt/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


Frys.com Open
CIMB Classic
WGC-HSBC Champions
(*) Wendy's 3-Tour Challenge
(*) Franklin Templeton
Hyundai T of C
Humana Challenge
Farmers Insurance Open
Waste Management Phoenix
WGC-Accenture Match Play
The Honda Classic
WGC-Cadillac Championship
Arnold Palmer Invt'l
Valero Texas Open
Masters Tournament
RBC Heritage
Zurich Classic of New Orl
THE PLAYERS Championship
the Memorial Tournament
FedEx St. Jude Classic
U.S. Open
(*) CVS Caremark Charity
Quicken Loans National
The Open Championship
PGA Championship
Wyndham Championship
The Barclays
Deutsche Bank Champ
BMW Championship
TOUR Championship
Shriners Hospitals
The McGladrey Classic
Sony Open in Hawaii
AT&T Pebble Beach
Shell Houston Open
Wells Fargo Championship
Crowne Plaza Invitational
The Greenbrier Classic
John Deere Classic
WGC-Bridgestone Invit'l
(*) World Challenge
(*) Ryder Cup
Northern Trust Open
Valspar Championship
RBC Canadian Open
Travelers Championship
HP Byron Nelson Champ
(*) World Cup of Golf
(*) PGA Grand Slam
OHL 

OperationalError: table "(*) ISPS Handa World Cup" already exists

<IPython.core.display.Javascript object>

In [131]:
conn.close()

<IPython.core.display.Javascript object>